# Cel: Przewidywanie spalenia kalorii w trakcie treningu cardio

In [141]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from helpers import time_to_seconds, calc_avg_speed

### Import raw dataset  
*Dataset imported from Garmin Connect application*

In [142]:
df_activities = pd.read_csv("activities_bronze.csv")

* Display data

In [143]:
df_activities

,Typ aktywności,Data,Ulubiony,Tytuł,Dystans,Suma kalorii,Czas,Średnie tętno,Maksymalne tętno,Aerobowy TE,Średnia kadencja pedałowania,Maksymalna kadencja pedałowania,Średnia prędkość,Maksymalna prędkość,Całkowity wznios,Całkowity spadek,Średnia długość kroku,Średnie odchylenie do długości,Średnie odchylenie pionowe,Średni czas kontaktu z podłożem,Średni GAP,Normalized Power® (NP®),Training Stress Score® (TSS®),Średnia moc,Maksymalna moc,Kroki,Razem powtórzeń,Razem serii,Utrata Body Battery,Dekompresja,Czas najlepszego okrążenia,Liczba okrążeń,Średnia częstotliwość oddechu,Minimalna częstotliwość oddechu,Maksymalna częstotliwość oddechu,Zmiana poziomu stresu,Początek stresu,Koniec stresu,Śr. Stres,Maksymalny poziom stresu,Czas ruchu,Upłynęło czasu,Minimalna wysokość,Maksymalna wysokość
0,Trening siłowy,2025-10-27 06:30:07,False,Siła,0.00,115,0:31,96,117,0.4,--,--,--,--,--,--,--,--,--,--,--,--,0.0,--,--,--,--,1,-4,Nie,00:30:28,1,--,--,--,--,--,--,--,--,00:30:28,00:32:11,--,--
1,Bieganie,2025-10-24 17:45:37,False,Poznań Bieganie,7.02,437,0:41,161,184,4.3,169,223,5:48,4:04,31,33,1.01,8.8,8.8,256,5:50,260,0.0,256,378,"6,926",--,--,-12,Nie,00:00:00.4,10,--,--,--,--,--,--,--,--,00:40:16,00:44:06,51,68
2,Kolarstwo,2025-10-23 14:41:35,False,Poznań Kolarstwo,4.00,85,0:20,101,130,0.6,--,--,12.5,25.2,26,21,--,--,--,--,--,--,0.0,--,--,--,--,--,-5,Nie,00:00:24.1,5,--,--,--,--,--,--,--,--,00:18:22,01:18:21,56,74
3,Trening siłowy,2025-10-23 07:25:29,False,Siła,0.00,89,0:22,98,124,0.4,--,--,--,--,--,--,--,--,--,--,--,--,0.0,--,--,--,--,1,-3,Nie,00:21:51,1,--,--,--,--,--,--,--,--,00:21:51,00:23:24,--,--
4,Kolarstwo,2025-10-22 07:26:43,False,Poznań Kolarstwo,6.98,218,0:34,120,156,2.0,--,--,12.4,26.5,34,41,--,--,--,--,--,--,0.0,--,--,--,--,--,-8,Nie,00:03:08.7,7,--,--,--,--,--,--,--,--,00:29:28,01:23:24,55,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Kolarstwo,2023-04-22 15:54:40,False,Poznań Kolarstwo,17.08,304,1:13,--,--,--,--,--,14.2,27.2,111,107,--,--,--,--,--,--,0.0,--,--,--,--,--,--,Nie,00:00:00.7,2,--,--,--,--,--,--,--,--,01:09:02,01:20:41,54,98
428,Kolarstwo,2023-04-22 14:35:57,False,Poznań Kolarstwo,14.42,282,1:12,--,--,--,--,--,12.2,25.1,147,152,--,--,--,--,--,--,0.0,--,--,--,--,--,--,Nie,00:00:00.9,2,--,--,--,--,--,--,--,--,01:03:57,01:18:36,44,96
429,Chodzenie,2023-04-21 09:53:01,False,Chodzenie,3.28,182,0:45,--,--,--,--,--,13:39,6:24,--,--,--,--,--,--,--,--,0.0,--,--,--,--,--,--,Nie,00:00:00.5,2,--,--,--,--,--,--,--,--,00:33:06,00:45:56,--,--
430,Bieganie,2023-04-21 06:38:44,False,Poznań Bieganie,3.04,177,0:22,--,--,--,--,--,7:09,3:19,30,32,--,--,--,--,--,--,0.0,--,--,--,--,--,--,Nie,00:00:00.4,2,--,--,--,--,--,--,--,--,00:21:00,00:21:44,50,71


* Set display.max_columns option, to increase readability of dataframe

In [144]:
pd.set_option('display.max_columns', None)

* Display columns and remove obsolete ones

In [145]:
df_activities.columns

Index(['Typ aktywności', 'Data', 'Ulubiony', 'Tytuł', 'Dystans',
       'Suma kalorii', 'Czas', 'Średnie tętno', 'Maksymalne tętno',
       'Aerobowy TE', 'Średnia kadencja pedałowania',
       'Maksymalna kadencja pedałowania', 'Średnia prędkość',
       'Maksymalna prędkość', 'Całkowity wznios', 'Całkowity spadek',
       'Średnia długość kroku', 'Średnie odchylenie do długości',
       'Średnie odchylenie pionowe', 'Średni czas kontaktu z podłożem',
       'Średni GAP', 'Normalized Power® (NP®)',
       'Training Stress Score® (TSS®)', 'Średnia moc', 'Maksymalna moc',
       'Kroki', 'Razem powtórzeń', 'Razem serii', 'Utrata Body Battery',
       'Dekompresja', 'Czas najlepszego okrążenia', 'Liczba okrążeń',
       'Średnia częstotliwość oddechu', 'Minimalna częstotliwość oddechu',
       'Maksymalna częstotliwość oddechu', 'Zmiana poziomu stresu',
       'Początek stresu', 'Koniec stresu', 'Śr. Stres',
       'Maksymalny poziom stresu', 'Czas ruchu', 'Upłynęło czasu',
       'Minim

In [146]:
df_activities = df_activities[['Typ aktywności', 'Data', 'Dystans',
       'Suma kalorii', 'Średnie tętno', 'Średnia kadencja pedałowania',
       'Średnia prędkość', 'Całkowity wznios', 'Całkowity spadek',
       'Średnia długość kroku', 'Kroki', 'Utrata Body Battery',
       'Czas ruchu', 'Minimalna wysokość', 'Maksymalna wysokość']]

* Normalize column names - replace space with underscore, lower text and remove polish special characters

In [147]:
columns = df_activities.columns
new_columns = [column.replace(" ", "_").lower().replace("ś", 's').replace("ł", "l").replace("ę", "e").replace("ć", "c") for column in columns]

for column, new_column in zip(columns, new_columns):
    df_activities.rename(columns={column: new_column}, inplace=True)


/var/folders/kf/w1zznf913ql0wqdsd3vdwp200000gn/T/ipykernel_17836/1668819079.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities.rename(columns={column: new_column}, inplace=True)


* Explore data types and dataframe shape

In [148]:
df_activities.dtypes

typ_aktywnosci                  object
data                            object
dystans                         object
suma_kalorii                    object
srednie_tetno                   object
srednia_kadencja_pedalowania    object
srednia_predkosc                object
calkowity_wznios                object
calkowity_spadek                object
srednia_dlugosc_kroku           object
kroki                           object
utrata_body_battery             object
czas_ruchu                      object
minimalna_wysokosc              object
maksymalna_wysokosc             object
dtype: object

In [149]:
df_activities.shape

(432, 15)

* Narrow data only to cardio activities

In [150]:
df_activities['typ_aktywnosci'].unique()

array(['Trening siłowy', 'Bieganie', 'Kolarstwo',
       'Jazda na rowerze treningowym', 'Chodzenie', 'Bieg na bieżni',
       'Pilates', 'Oddech', 'Piesze wędrówki', 'Joga', 'Pływanie', 'Inne',
       'Narciarstwo w kurorcie'], dtype=object)

In [151]:
df_activities = df_activities[df_activities['typ_aktywnosci'].isin(['Bieganie', 'Kolarstwo',
       'Jazda na rowerze treningowym', 'Chodzenie', 'Bieg na bieżni', 'Piesze wędrówki'])]

* Verify dataframe dimension

In [152]:
df_activities.shape

(260, 15)

* Look for NaN values
    * no NaN because missing values represented as "--"
    * replace "--" with NaN


In [153]:
df_activities.isna().sum()

typ_aktywnosci                  0
data                            0
dystans                         0
suma_kalorii                    0
srednie_tetno                   0
srednia_kadencja_pedalowania    0
srednia_predkosc                0
calkowity_wznios                0
calkowity_spadek                0
srednia_dlugosc_kroku           0
kroki                           0
utrata_body_battery             0
czas_ruchu                      0
minimalna_wysokosc              0
maksymalna_wysokosc             0
dtype: int64

In [154]:
df_activities.replace('--', np.nan, inplace=True)

* Display DF to look for other data issues

In [155]:
df_activities

,typ_aktywnosci,data,dystans,suma_kalorii,srednie_tetno,srednia_kadencja_pedalowania,srednia_predkosc,calkowity_wznios,calkowity_spadek,srednia_dlugosc_kroku,kroki,utrata_body_battery,czas_ruchu,minimalna_wysokosc,maksymalna_wysokosc
1,Bieganie,2025-10-24 17:45:37,7.02,437,161,169,5:48,31,33,1.01,"6,926",-12,00:40:16,51,68
2,Kolarstwo,2025-10-23 14:41:35,4.00,85,101,NaN,12.5,26,21,NaN,NaN,-5,00:18:22,56,74
4,Kolarstwo,2025-10-22 07:26:43,6.98,218,120,NaN,12.4,34,41,NaN,NaN,-8,00:29:28,55,82
5,Jazda na rowerze treningowym,2025-10-21 17:58:41,0.00,486,145,NaN,NaN,NaN,NaN,NaN,NaN,-10,00:00:00,NaN,NaN
6,Chodzenie,2025-10-20 12:03:19,3.01,169,86,89,16:33,NaN,NaN,0.67,"5,032",-5,00:42:39,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Chodzenie,2023-04-24 13:48:06,5.04,226,NaN,NaN,15:49,NaN,NaN,NaN,NaN,NaN,00:55:47,NaN,NaN
427,Kolarstwo,2023-04-22 15:54:40,17.08,304,NaN,NaN,14.2,111,107,NaN,NaN,NaN,01:09:02,54,98
428,Kolarstwo,2023-04-22 14:35:57,14.42,282,NaN,NaN,12.2,147,152,NaN,NaN,NaN,01:03:57,44,96
429,Chodzenie,2023-04-21 09:53:01,3.28,182,NaN,NaN,13:39,NaN,NaN,NaN,NaN,NaN,00:33:06,NaN,NaN


* Remove commas from suma_kalorii, kroki, neccessary for type casting


In [156]:
df_activities.loc[:, ['suma_kalorii', 'kroki']] = df_activities[['suma_kalorii', 'kroki']].map(lambda x: str(x).replace(',', ''))

* Verify datatypes

In [157]:
df_activities.dtypes

typ_aktywnosci                  object
data                            object
dystans                         object
suma_kalorii                    object
srednie_tetno                   object
srednia_kadencja_pedalowania    object
srednia_predkosc                object
calkowity_wznios                object
calkowity_spadek                object
srednia_dlugosc_kroku           object
kroki                           object
utrata_body_battery             object
czas_ruchu                      object
minimalna_wysokosc              object
maksymalna_wysokosc             object
dtype: object

* Convert numeric columns to numeric

In [158]:
numeric_cols = ['dystans', 'suma_kalorii',
       'srednie_tetno', 'srednia_kadencja_pedalowania', 'srednia_predkosc',
       'calkowity_wznios', 'calkowity_spadek', 'srednia_dlugosc_kroku',
       'kroki', 'utrata_body_battery', 'minimalna_wysokosc',
       'maksymalna_wysokosc']

for col in numeric_cols:
    df_activities[col] = pd.to_numeric(df_activities[col], errors='coerce')

In [159]:
df_activities.dtypes

typ_aktywnosci                   object
data                             object
dystans                         float64
suma_kalorii                      int64
srednie_tetno                   float64
srednia_kadencja_pedalowania    float64
srednia_predkosc                float64
calkowity_wznios                float64
calkowity_spadek                float64
srednia_dlugosc_kroku           float64
kroki                           float64
utrata_body_battery             float64
czas_ruchu                       object
minimalna_wysokosc              float64
maksymalna_wysokosc             float64
dtype: object

* Convert activity time column to seconds using *time_to_seconds* function from helper.py

In [160]:
df_activities['czas_ruchu'] = df_activities['czas_ruchu'].apply(time_to_seconds)

In [161]:
df_activities.dtypes

typ_aktywnosci                   object
data                             object
dystans                         float64
suma_kalorii                      int64
srednie_tetno                   float64
srednia_kadencja_pedalowania    float64
srednia_predkosc                float64
calkowity_wznios                float64
calkowity_spadek                float64
srednia_dlugosc_kroku           float64
kroki                           float64
utrata_body_battery             float64
czas_ruchu                      float64
minimalna_wysokosc              float64
maksymalna_wysokosc             float64
dtype: object

* Convert datetime column to datetime type

In [162]:
df_activities['data'] = pd.to_datetime(df_activities['data'])

In [163]:
df_activities.dtypes

typ_aktywnosci                          object
data                            datetime64[ns]
dystans                                float64
suma_kalorii                             int64
srednie_tetno                          float64
srednia_kadencja_pedalowania           float64
srednia_predkosc                       float64
calkowity_wznios                       float64
calkowity_spadek                       float64
srednia_dlugosc_kroku                  float64
kroki                                  float64
utrata_body_battery                    float64
czas_ruchu                             float64
minimalna_wysokosc                     float64
maksymalna_wysokosc                    float64
dtype: object

*  Unify activities type to main 3 groups

In [164]:
df_activities['typ_aktywnosci'].unique() 

array(['Bieganie', 'Kolarstwo', 'Jazda na rowerze treningowym',
       'Chodzenie', 'Bieg na bieżni', 'Piesze wędrówki'], dtype=object)

In [165]:
map_activities = {
    'Bieganie': 'bieganie',
    'Bieg na bieżni': 'bieganie',
    'Kolarstwo': 'kolarstwo',
    'Jazda na rowerze treningowym': 'kolarstwo',
    'Chodzenie': 'chodzenie',
    'Piesze wędrówki': 'chodzenie'
}

df_activities['typ_aktywnosci'] = df_activities['typ_aktywnosci'].replace(map_activities)

In [166]:
df_activities

,typ_aktywnosci,data,dystans,suma_kalorii,srednie_tetno,srednia_kadencja_pedalowania,srednia_predkosc,calkowity_wznios,calkowity_spadek,srednia_dlugosc_kroku,kroki,utrata_body_battery,czas_ruchu,minimalna_wysokosc,maksymalna_wysokosc
1,bieganie,2025-10-24 17:45:37,7.02,437,161.0,169.0,NaN,31.0,33.0,1.01,6926.0,-12.0,2416.0,51.0,68.0
2,kolarstwo,2025-10-23 14:41:35,4.00,85,101.0,NaN,12.5,26.0,21.0,NaN,NaN,-5.0,1102.0,56.0,74.0
4,kolarstwo,2025-10-22 07:26:43,6.98,218,120.0,NaN,12.4,34.0,41.0,NaN,NaN,-8.0,1768.0,55.0,82.0
5,kolarstwo,2025-10-21 17:58:41,0.00,486,145.0,NaN,NaN,NaN,NaN,NaN,NaN,-10.0,0.0,NaN,NaN
6,chodzenie,2025-10-20 12:03:19,3.01,169,86.0,89.0,NaN,NaN,NaN,0.67,5032.0,-5.0,2559.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,chodzenie,2023-04-24 13:48:06,5.04,226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3347.0,NaN,NaN
427,kolarstwo,2023-04-22 15:54:40,17.08,304,NaN,NaN,14.2,111.0,107.0,NaN,NaN,NaN,4142.0,54.0,98.0
428,kolarstwo,2023-04-22 14:35:57,14.42,282,NaN,NaN,12.2,147.0,152.0,NaN,NaN,NaN,3837.0,44.0,96.0
429,chodzenie,2023-04-21 09:53:01,3.28,182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1986.0,NaN,NaN


* Verify NaN values

In [167]:
df_activities.isna().sum()

typ_aktywnosci                    0
data                              0
dystans                           0
suma_kalorii                      0
srednie_tetno                    17
srednia_kadencja_pedalowania    243
srednia_predkosc                238
calkowity_wznios                 45
calkowity_spadek                 47
srednia_dlugosc_kroku           243
kroki                           161
utrata_body_battery             233
czas_ruchu                        0
minimalna_wysokosc               55
maksymalna_wysokosc              47
dtype: int64

* Remove unnecessary columns

In [168]:
# Remove unnecessary columns
df_activities = df_activities[['typ_aktywnosci', 'dystans', 'suma_kalorii', 'srednie_tetno', 'srednia_predkosc', 'czas_ruchu']]

In [169]:
df_activities.isna().sum()

typ_aktywnosci        0
dystans               0
suma_kalorii          0
srednie_tetno        17
srednia_predkosc    238
czas_ruchu            0
dtype: int64

* Imput values for column 'srednia_predkosc'. Value is calculated based on columns dystans and czas_ruchu, if present. Using *calc_avg_speed* function from helpers.py

In [170]:
df_activities['srednia_predkosc'] = df_activities.apply(lambda row: calc_avg_speed(row['czas_ruchu'], row['dystans']), axis=1)

/var/folders/kf/w1zznf913ql0wqdsd3vdwp200000gn/T/ipykernel_17836/3244018590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities['srednia_predkosc'] = df_activities.apply(lambda row: calc_avg_speed(row['czas_ruchu'], row['dystans']), axis=1)


In [171]:
df_activities.isna().sum()

typ_aktywnosci       0
dystans              0
suma_kalorii         0
srednie_tetno       17
srednia_predkosc    24
czas_ruchu           0
dtype: int64

* Create new column 'aktywnosc_stacjonarna' that indicates if activity is outdoor or indoor

In [172]:
df_activities['aktywnosc_stacjonarna'] = np.where((df_activities['srednia_predkosc'].isna() | df_activities['srednie_tetno'].isna()), 1, 0)

/var/folders/kf/w1zznf913ql0wqdsd3vdwp200000gn/T/ipykernel_17836/3917759937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities['aktywnosc_stacjonarna'] = np.where((df_activities['srednia_predkosc'].isna() | df_activities['srednie_tetno'].isna()), 1, 0)


In [173]:
df_activities

,typ_aktywnosci,dystans,suma_kalorii,srednie_tetno,srednia_predkosc,czas_ruchu,aktywnosc_stacjonarna
1,bieganie,7.02,437,161.0,10.46,2416.0,0
2,kolarstwo,4.00,85,101.0,13.07,1102.0,0
4,kolarstwo,6.98,218,120.0,14.21,1768.0,0
5,kolarstwo,0.00,486,145.0,NaN,0.0,1
6,chodzenie,3.01,169,86.0,4.23,2559.0,0
...,...,...,...,...,...,...,...
425,chodzenie,5.04,226,NaN,5.42,3347.0,1
427,kolarstwo,17.08,304,NaN,14.85,4142.0,1
428,kolarstwo,14.42,282,NaN,13.53,3837.0,1
429,chodzenie,3.28,182,NaN,5.95,1986.0,1


* Verify current dimension and missing values

In [174]:
df_activities.isna().sum()

typ_aktywnosci            0
dystans                   0
suma_kalorii              0
srednie_tetno            17
srednia_predkosc         24
czas_ruchu                0
aktywnosc_stacjonarna     0
dtype: int64

In [175]:
df_activities.shape

(260, 7)

* Imput 0.0 in 'srednia_predkosc' column, as this data was not collected in raw data. Model will recognize it and will learn that 0.0 and aktywnosc_stacjonarna == 1, means that activity should be calculated based on time and hr

In [176]:
df_activities['srednia_predkosc'] = df_activities['srednia_predkosc'].fillna(0.0).astype(float)

/var/folders/kf/w1zznf913ql0wqdsd3vdwp200000gn/T/ipykernel_17836/3844927609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities['srednia_predkosc'] = df_activities['srednia_predkosc'].fillna(0.0).astype(float)


* Imput 'srednie_tetno' median for each activity group

In [177]:
mediany_tetna = df_activities.groupby('typ_aktywnosci')['srednie_tetno'].median()
mediany_tetna

typ_aktywnosci
bieganie     166.0
chodzenie     99.0
kolarstwo    132.5
Name: srednie_tetno, dtype: float64

In [178]:
df_activities['srednie_tetno'] = df_activities.groupby('typ_aktywnosci')['srednie_tetno'].transform(lambda x: x.fillna(x.median()))

/var/folders/kf/w1zznf913ql0wqdsd3vdwp200000gn/T/ipykernel_17836/1143395750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities['srednie_tetno'] = df_activities.groupby('typ_aktywnosci')['srednie_tetno'].transform(lambda x: x.fillna(x.median()))


* Confirm missing values

In [179]:
df_activities.isna().sum()

typ_aktywnosci           0
dystans                  0
suma_kalorii             0
srednie_tetno            0
srednia_predkosc         0
czas_ruchu               0
aktywnosc_stacjonarna    0
dtype: int64

In [180]:
df_activities

,typ_aktywnosci,dystans,suma_kalorii,srednie_tetno,srednia_predkosc,czas_ruchu,aktywnosc_stacjonarna
1,bieganie,7.02,437,161.0,10.46,2416.0,0
2,kolarstwo,4.00,85,101.0,13.07,1102.0,0
4,kolarstwo,6.98,218,120.0,14.21,1768.0,0
5,kolarstwo,0.00,486,145.0,0.00,0.0,1
6,chodzenie,3.01,169,86.0,4.23,2559.0,0
...,...,...,...,...,...,...,...
425,chodzenie,5.04,226,99.0,5.42,3347.0,1
427,kolarstwo,17.08,304,132.5,14.85,4142.0,1
428,kolarstwo,14.42,282,132.5,13.53,3837.0,1
429,chodzenie,3.28,182,99.0,5.95,1986.0,1


* Export cleaned data to CSV file

In [181]:
df_activities.to_csv("activities_silver.csv")